# 🌿 Edge AI Prototype: Recyclable Item Classification
This notebook trains a lightweight image classification model using TensorFlow, converts it to TensorFlow Lite, and simulates inference for Edge AI deployment.

In [ ]:
# Step 1: Install TensorFlow and Import Libraries
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [ ]:
# Step 2: Load and Preprocess Dataset
# Replace with actual paths to your dataset (e.g., from Kaggle)
train_dir = '/content/train'
val_dir = '/content/val'

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(64, 64),
                                                    batch_size=32,
                                                    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='categorical')

In [ ]:
# Step 3: Build the CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Step 4: Train the Model
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=val_generator)

In [ ]:
# Step 5: Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('recyclable_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
# Step 6: Load TFLite Model and Run Inference
interpreter = tf.lite.Interpreter(model_path='recyclable_model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Dummy input (replace with a real preprocessed image for real inference)
input_shape = input_details[0]['shape']
input_data = np.random.random_sample(input_shape).astype(np.float32)

interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
print("Predicted class probabilities:", output_data)